In [1]:
import bokeh

In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [3]:
# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

In [4]:
x

[1, 2, 3, 4, 5]

In [5]:
# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness to the plot
p.line(x, y, legend_label="Temp.", line_width=2)

# show the results
show(p)

In [6]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS
import numpy as np

#output_notebook()
#
# Define initial data
mean_init = 0
stddev_init = 1
x = np.linspace(-10, 10, 400)
y = 1 / (stddev_init * np.sqrt(2 * np.pi)) * np.exp(-(x - mean_init)**2 / (2 * stddev_init**2))
source = ColumnDataSource(data={'x': x, 'y': y})

# Create a new plot
plot = figure(width=400, height=400)  # Adjust parameter names if needed
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

# JavaScript callback for sliders
callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const A = cb_obj.value;
    const x = data['x'];
    const y = data['y'];
    const B = stddev_slider.value;
    for (let i = 0; i < x.length; i++) {
        y[i] = 1 / (B * Math.sqrt(2 * Math.PI)) * Math.exp(-Math.pow(x[i] - A, 2) / (2 * Math.pow(B, 2)));
    }
    source.change.emit();
""")

mean_slider = Slider(start=-10, end=10, value=mean_init, step=0.1, title="Mean")
mean_slider.js_on_change('value', callback)

stddev_slider = Slider(start=0.1, end=5, value=stddev_init, step=0.1, title="Standard Deviation")
stddev_slider.js_on_change('value', callback)

callback.args["stddev_slider"] = stddev_slider  # Update callback to include stddev_slider

layout = column(mean_slider, stddev_slider, plot)

show(layout)


In [7]:
#!pip install ipympl

In [8]:
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure
from scipy.stats import norm
import numpy as np

#output_notebook()

# Standard normal curve data
x = np.linspace(-5, 5, 1000)
y = norm.pdf(x)

# Bokeh plot setup
plot = figure(title="Area under Standard Normal Curve", tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[-5, 5], y_range=[0, max(y)*1.1])

line = plot.line(x, y, line_width=2)
shaded_area = plot.varea(x, y1=0, y2=y, alpha=0.3)

# JavaScript callback for interactive sliders
callback = CustomJS(args=dict(line=line, shaded_area=shaded_area, x=x, y=y), code="""
    const data = line.data;
    const A = shaded_area.data;
    const left_limit = left_slider.value;
    const right_limit = right_slider.value;
    const y = data['y'];
    const x = data['x'];
    const newY = [];
    const newX = [];
    
    for (let i = 0; i < x.length; i++) {
        if (x[i] >= left_limit && x[i] <= right_limit) {
            newY.push(y[i]);
            newX.push(x[i]);
        } else {
            newY.push(0);
            newX.push(x[i]);
        }
    }
    
    A['y2'] = newY;
    shaded_area.change.emit();
""")

# Sliders for left and right limits
left_slider = Slider(start=-5, end=5, value=-1, step=0.1, title="Left Limit")
right_slider = Slider(start=-5, end=5, value=1, step=0.1, title="Right Limit")

left_slider.js_on_change('value', callback)
right_slider.js_on_change('value', callback)

callback.args["left_slider"] = left_slider
callback.args["right_slider"] = right_slider

layout = column(left_slider, right_slider, plot)
show(layout)


In [9]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column
from bokeh.models import CustomJS, Slider, Div
from bokeh.plotting import figure
from scipy.stats import norm
import numpy as np

#
# Standard normal curve data
x = np.linspace(-5, 5, 1000)
y = norm.pdf(x)

# Bokeh plot setup
plot = figure(title="Area under Standard Normal Curve", tools="crosshair,pan,reset,save,wheel_zoom",
              x_range=[-5, 5], y_range=[0, max(y)*1.1])

line = plot.line(x, y, line_width=2)
shaded_area = plot.varea(x, y1=0, y2=y, alpha=0.3)

# Area calculation and display
area_text = Div(text="Area: 0.00%")

def update_area(left, right):
    area = norm.cdf(right) - norm.cdf(left)
    area_text.text = f"Area: {area:.2%}"

# Sliders for left and right limits
left_slider = Slider(start=-5, end=5, value=-1, step=0.1, title="Left Limit")
right_slider = Slider(start=-5, end=5, value=1, step=0.1, title="Right Limit")

def update(attr, old, new):
    left = left_slider.value
    right = right_slider.value
    x_new = np.linspace(-5, 5, 1000)
    y_new = np.where((x_new >= left) & (x_new <= right), norm.pdf(x_new), 0)
    shaded_area.data_source.data['y2'] = y_new
    update_area(left, right)

left_slider.on_change('value', update)
right_slider.on_change('value', update)

layout = column(left_slider, right_slider, area_text, plot)

# Show plot
show(layout, notebook_handle=True)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

